# <font color='#eb3483'> Feature Selection and Regularization in Linear Regression</font>

Before working through this notebook, you should read Chapter 6, Sections 6.1 and 6.2 of [Introduction to Statistical Learning](https://static1.squarespace.com/static/5ff2adbe3fe4fe33db902812/t/6062a083acbfe82c7195b27d/1617076404560/ISLR%2BSeventh%2BPrinting.pdf). 


In [ ]:
import pandas as pd

## <font color='#eb3483'> Import and Explore the Data </font>

For this notebook, we'll be using the Hitters data that is provided as a csv file in the `data` folder.

We wish to predict a baseball player's salary on the basis of various statistics associated with performance in the previous year.

Let's import the data and take a look at it:

In [ ]:
df = pd.read_csv('data/Hitters.csv')
df.head()

Note that some `Salary` values seem to be `NaN`. Let's check for missing values across all the variables and remove the rows with missing values:

In [ ]:
df.isna().sum()

In [ ]:
print('Rows before = ', df.shape[0])
df.dropna(inplace=True)
print('Rows after = ', df.shape[0])

Let's also deal with those categorical variables:

In [ ]:
df.dtypes

In [ ]:
df.League.value_counts()

In [ ]:
df.Division.value_counts()

In [ ]:
df.NewLeague.value_counts()

In [ ]:
df['League'] = 1*(df.League=='A')
df['Division'] = 1*(df.Division=='W')
df['NewLeague'] = 1*(df.NewLeague=='A')
df.head()

Next, let's standardize each predictor; that is, we subtract the predictor's mean and divide by the predictor's standard deviation. This way, each column will be centred around zero and have the same scale. This is required for the regularization techniques that we will look at below.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(df[df.columns[df.columns!='Salary']])
X = pd.DataFrame(X, columns = df.columns[df.columns!='Salary'])
X.head()

Let's begin our analyses by fitting a standard multiple regression model to predict `Salary`:

In [ ]:
from sklearn.linear_model import LinearRegression
olsmod = LinearRegression()
olsmod.fit(X=X, y=df.Salary)
pd.Series(olsmod.coef_, index=X.columns).sort_values()

Because the features all have the same scale, we can tell from the coefficients which ones are most influential. Higher values of `CRuns`, `Hits` `CRBI` and `Walks` and lower values of `CAtBat`, `AtBat` and `CWalks` are strongly associated with a higher `Salary`.

Can we drop some of the intermediate features, those that don't seem to be strongly associated with `Salary`?

## <font color='#eb3483'> Recursive Feature Elimination </font>

In Section 6.1 of [Intro to Stat Learning]((https://static1.squarespace.com/static/5ff2adbe3fe4fe33db902812/t/6062a083acbfe82c7195b27d/1617076404560/ISLR%2BSeventh%2BPrinting.pdf)), the authors describe stepwise and best subset selection. These are not implemented in scikit-learn. However, scikit-learn does have a procedure called [Recursive Feature Elimination (RFE)](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html) that does feature selection. You can read more about it through the link above. In a nutshell, the procedure begins by fitting a model with all the features and then recursively removes them from the model based on how much they contribute to model performance (the worst contributors are eliminated first). This is very similar to the *backward stepwise* procedure described in Section 6.1. 

RFE is available via the `RFE` function from the `feature_selection` submodule in `sklearn`. This function requires that you specify the number of features that you would like in your model, say, 5 and it then determines which 5 features are the best. Alternatively, you can use the `RFECV` function to determine the optimal *number* of features as well. Note that this function can be applied to any `sklearn` estimator, though we will only consider linear regression below.

In [ ]:
from sklearn.feature_selection import RFECV
?RFECV

In [ ]:
rfe = RFECV(estimator=olsmod, cv=5, scoring="neg_mean_squared_error")
rfe.fit(X=X, y=df.Salary)
print('Selected', rfe.n_features_, 'features:')
X.columns[rfe.support_]

Note that these are the features with the largest coefficients (in absolute value) in our full model above.

## <font color='#eb3483'> Ridge Regression </font>

Section 6.2 of [Intro to Stat Learning]((https://static1.squarespace.com/static/5ff2adbe3fe4fe33db902812/t/6062a083acbfe82c7195b27d/1617076404560/ISLR%2BSeventh%2BPrinting.pdf)) describes ridge regression. In short, ridge regression uses a different loss function to OLS regression. Recall that OLS regression finds the coefficients that minimise the residual sum of squares (RSS) loss. Ridge regression adds a penalty term to this to prevent coefficients from getting too big. The extent to which this penalty plays a role is determined by a hyperparameter $\lambda$. When $\lambda = 0$, we get OLS regression (no penalty). Larger values of $\lambda$ lead to more *regularization* - the coefficients get shrunk toward zero. You can see this as making the model simpler. Thus increasing $\lambda$ increases bias and reduces variance (the bias-variance trade-off!). As always, to choose the best compromise between bias and variance (and hence the best $\lambda$), we use cross validation. 

Note that $\lambda$ is called "alpha" in the `sklearn` implementation.

In [ ]:
from sklearn.linear_model import RidgeCV
?RidgeCV

In [ ]:
ridge = RidgeCV(alphas=[0.1, 1.0, 10.0], scoring="neg_mean_squared_error", cv=5)
ridge.fit(X=X, y=df.Salary)

In [ ]:
ridge.alpha_ # this is the best value of alpha based on CV

In [ ]:
pd.DataFrame({'ols':olsmod.coef_, 'ridge':ridge.coef_}, index=X.columns)
# notice how the ridge regression coefficients are often (not always) pulled closer to zero

## <font color='#eb3483'> Lasso Regression </font>

Finally, Section 6.2 of [Intro to Stat Learning]((https://static1.squarespace.com/static/5ff2adbe3fe4fe33db902812/t/6062a083acbfe82c7195b27d/1617076404560/ISLR%2BSeventh%2BPrinting.pdf)) also describes lasso regression or "the lasso". Conceptually, the lasso is very similar to ridge regression - it adds a penality to the residual sum of squares to rein in the regression coefficients. It also has a hyperparameter $\lambda$ that controls the influence of the penalty (the bias-variance trade-off). The main difference is that the mathematical form of the lasoo penalty allows the lasso to set some coefficients to exactly zero i.e. it can perform feature elimination. 

In [ ]:
from sklearn.linear_model import LassoCV
?LassoCV

In [ ]:
lasso = LassoCV(n_alphas=100, cv=5)
lasso.fit(X=X, y=df.Salary)

In [ ]:
lasso.alpha_ # this is best value of lambda based on CV

In [ ]:
pd.Series(lasso.coef_, index=X.columns) 
# notice how many of these have been set to 0 (dropped from the mode)
# this is a special property of the lasso penalty

In [ ]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import cross_val_score

ridge_best = Ridge(alpha=ridge.alpha_)
lasso_best = Lasso(alpha=lasso.alpha_)

cv_ols = cross_val_score(estimator=olsmod, X=X, y=df.Salary, scoring="neg_mean_squared_error", cv=5)
cv_rfe = cross_val_score(estimator=olsmod, X=X[X.columns[rfe.support_]], y=df.Salary, scoring="neg_mean_squared_error", cv=5)
cv_ridge = cross_val_score(estimator=ridge_best, X=X, y=df.Salary, scoring="neg_mean_squared_error", cv=5)
cv_lasso = cross_val_score(estimator=lasso_best, X=X, y=df.Salary, scoring="neg_mean_squared_error", cv=5)

print('OLS MSE:', -cv_ols.mean())
print('RFE MSE:', -cv_rfe.mean())
print('Ridge MSE:', -cv_ridge.mean())
print('Lasso MSE:', -cv_lasso.mean())

In this example, it looks like RFE is the best procedure!